In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sp
from scipy.special import jn_zeros
from scipy.special import gamma
from scipy.special import factorial
from scipy.optimize import fsolve
from scipy.optimize import brentq

In [36]:
def j_coeff(m,x_i,n,x):
    return((-1)**m)/(factorial(m)*gamma(m+n+1))*(x[x_i]/2)**(2*m+n)
def J(x,n,N):
    array_to_sum = np.fromfunction(lambda m,x_i: j_coeff(m,x_i,n,x), (N,x.shape[0]),dtype="int")
    array_to_sum = np.nan_to_num(array_to_sum, nan=0.0)
    return(sum(array_to_sum))


def find_zeros(N_zeros,n,N):
    zeros = []
    Max = N_zeros*1000
    count = 0
    init_start= 0.01
    init_end = 1
    zero = -1
    while (len(zeros) < N_zeros) and (count< Max):
        count += 1
        try:
            zero = brentq(lambda x : J(np.array([x]),n,N)[0],init_start, init_end)
        except:
            init_end += 0.1
        if zero >=0:
            if len(zeros) == 0:
                zeros.append(zero)
                init_start = zero + 0.1
            elif np.abs(zeros[-1] - zero) > 0.1:
                zeros.append(zero)
                init_start = zero + 0.1

        
        
    return(zeros)


N=100
x = np.array([2.40482556,5.52007811,  8.65372791, 11.79153444, 14.93091771])
n=0
print(J(x,n,N))
print(J(x,n,N))
print('====================')
print(brentq(lambda x : J(np.array([x]),n,N)[0],0,5))
print('====================')
print(find_zeros(5,0,N))
print(find_zeros(5,1,N))
print(jn_zeros(0,5))
print(jn_zeros(1,5))

[-1.19623367e-09 -9.74217705e-11  7.90169973e-10  2.29786160e-10
 -3.11757715e-10]
[-1.19623367e-09 -9.74217705e-11  7.90169973e-10  2.29786160e-10
 -3.11757715e-10]
2.4048255576957724
[2.404825557695773, 5.520078110286303, 8.653727912911052, 11.791534439013276, 14.930917708503442]
[3.831705970207512, 7.015586669815596, 10.17346813506263, 13.323691936317086, 16.470630050941157]
[ 2.40482556  5.52007811  8.65372791 11.79153444 14.93091771]
[ 3.83170597  7.01558667 10.17346814 13.32369194 16.47063005]


/var/folders/my/2p0xcfbn4z100hsfx3lhh76m0000gp/T/ipykernel_983/3573234530.py:2: RuntimeWarning: overflow encountered in multiply
  return((-1)**m)/(factorial(m)*gamma(m+n+1))*(x[x_i]/2)**(2*m+n)


In [13]:
def f(x,y):
    return(np.exp(-x*y))

def Forward_Eular(x_n,y_n,f,h):
    return y_n + h* f(x_n,y_n)

def second_order_RK(x_n,y_n,f,h):
    k_1 = h*f(x_n,y_n)
    k_2 = h*f(x_n+h/2,y_n+k_1/2)
    return y_n + k_2

def fourth_order_RK(x_n,y_n,f,h):
    k_1 = f(x_n,y_n)
    k_2 = f(x_n+h/2,y_n+k_1*h/2)
    k_3 = f(x_n+h/2,y_n+k_2*h/2)
    k_4 = f(x_n+h,y_n+h*k_3)
    return y_n + h*(k_1+k_4)/6 +h*(k_2+k_3)/3

def run_solvers(h):
    x = 0
    n = int(1/h)
    Xs = np.zeros(n)
    FE = np.zeros(n)
    SORK = np.zeros(n)
    FORK = np.zeros(n)
    for i in range(n):
        i+1
        FE[i]   = Forward_Eular(  x, FE[i-1] ,f,h)
        SORK[i] = second_order_RK(x,SORK[i-1],f,h)
        FORK[i] = fourth_order_RK(x,FORK[i-1],f,h)
        Xs[i] = x
        x += h
    # plt.scatter(Xs,FE, label = "FE",s=150)
    # plt.scatter(Xs,SORK, label = "SORK",s=50)
    # plt.scatter(Xs,FORK, label = "FORK",s=1)
    # print(FE[-1])
    # print(SORK[-1])
    # print(FORK[-1])
    # print("0.773877305")
    # plt.show()
    return((FE[-1] - 0.773877305, SORK[-1] - 0.773877305, FORK[-1] - 0.773877305))

# evals = np.zeros((3,3))
# index = np.zeros(3,dtype=int)
# thresholds = [10**(-3),10**(-6),10**(-9)]
# eval_nums  = [1,2,3]
# for i in range(1000000):
#     print(index)
#     print(i)
#     if np.sum(index) > 9:
#         break
#     i+=1
#     h = 1/i
#     solutions = np.abs(run_solvers(h))
#     for j in range(3):
#         if solutions[j] < thresholds[index[j]]:
#             evals[j][index[j]] = i*eval_nums[j]
#             index[j] += 1
# print(evals)
run_solvers(1/1000000)

(2.2516404929451284e-07, -4.1284387020112945e-10, -4.1284331508961714e-10)